In [1]:
import pandas as pd
import hddm

rawData = hddm.load_csv('study1nonadaptive.csv')
data = rawData[rawData['sure'] == 0]
data['choice'] = data['choice'] ^ 1
data['loss'] = data['loss'] * (-1)
data.rename(columns = {'id':'subj_idx', 'choice':'response', 'win':'gain'}, inplace = True)

/anaconda3/lib/python2.7/site-packages/IPython/parallel.py:13: ShimWarning: The `IPython.parallel` package has been deprecated since IPython 4.0. You should import from ipyparallel instead.
  "You should import from ipyparallel instead.", ShimWarning)
/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python2.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/anaconda3/lib/python2.7/site-packages/pandas/core/frame.py:40

In [4]:
data.head()

,subj_idx,trial,rt,response,gain,loss,sure
1,1,2,3.790270,1,12.0,-7.50,0.0
3,1,4,2.003780,1,10.0,-15.00,0.0
4,1,5,3.993464,0,2.0,-3.00,0.0
7,1,8,4.908444,0,10.0,-12.50,0.0
8,1,9,1.064644,1,12.0,-3.75,0.0


In [5]:
m = hddm.models.HDDMRegressor(data, ['v ~ gain + loss - 1'], include=('z'), group_only_regressors=False)

Adding these covariates:
['v_gain', 'v_loss']


In [6]:
m.find_starting_values()

/anaconda3/lib/python2.7/site-packages/scipy/optimize/optimize.py:1934: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/anaconda3/lib/python2.7/site-packages/scipy/optimize/optimize.py:1935: RuntimeWarning: invalid value encountered in double_scalars
  p = (x - v) * tmp2 - (x - w) * tmp1


In [7]:
import os
modelName = 'no_alpha'
if not os.path.exists('savedModels/{}'.format(modelName)):
    os.makedirs('savedModels/{}'.format(modelName))

m.sample(5000, burn=1000, dbname='savedModels/{}/db_{}'.format(modelName, modelName), db='pickle')

 [-----------------100%-----------------] 5001 of 5000 complete in 10142.3 sec

In [8]:
m.save('savedModels/{}/{}'.format(modelName, modelName))

In [3]:
# modelName = 'no_alpha'
# m = hddm.load('savedModels/{}/{}'.format(modelName, modelName))

IOError: [Errno 2] No such file or directory: 'savedModels/no_alpha/no_alpha'

## Model parameters

In [6]:
m.print_stats()
m.print_stats(fname='savedModels/{}/{}_stats.txt'.format(modelName, modelName))

                      mean         std         2.5q          25q         50q         75q       97.5q       mc err
a                  2.69335    0.106301      2.48966      2.62335     2.69344     2.76248     2.90616   0.00165662
a_std             0.636498   0.0846914     0.499819     0.576241    0.627243     0.68735    0.828734   0.00186797
a_subj.1           2.37009    0.105265      2.16522       2.2971     2.36891     2.44232     2.57638   0.00312148
a_subj.2           2.09391   0.0746258       1.9531      2.04186      2.0921     2.14296     2.24371   0.00198203
a_subj.3           2.87124    0.103145      2.67326      2.79981     2.87038     2.94102     3.07763   0.00229446
a_subj.4           2.03744   0.0927627      1.86892      1.97283     2.03259     2.09733     2.23463   0.00200103
a_subj.5           2.22412   0.0770021      2.07684      2.17077     2.22417     2.27549     2.37864   0.00168036
a_subj.6           2.26501   0.0724332      2.12403      2.21782     2.26287     2.31238

In [7]:
modelParamsDF = m.nodes_db
modelParamsDF.to_csv('savedModels/{}/{}_nodes.csv'.format(modelName, modelName))

## Simulate data

In [9]:
ppc_data = hddm.utils.post_pred_gen(m, samples=50, progress_bar=True)

 [------------------107%------------------] 42 of 39 complete in 10767.5 sec

In [10]:
import numpy as np
dataArray = np.array(data)

def extractStakesFromParticipantIndex(participantIndex):
    participantData = dataArray[dataArray[:, 0] == participantIndex][:]
    participantStakes = participantData[:, 4:6]
    return participantStakes

In [11]:
import numpy as np

overallData = np.zeros((1, 5))

for (node, sample), sim_data in ppc_data.groupby(level=(0, 1)):
    print("-------")
    print("node: ", node)
    sim_data_array = np.asarray(sim_data)
    numTrials = np.shape(sim_data_array)[0]
    print("number of trials: ", numTrials)
    participantIndex = int(filter(str.isdigit, node))
    participantStakes = extractStakesFromParticipantIndex(participantIndex)
    participantIndexArray = np.full((numTrials, 1), participantIndex)
    responsesArray = np.zeros((numTrials, 1))
    responsesArray[sim_data_array[:, 0] > 0] = 1
    reactionTimesArray = np.abs(sim_data_array)
    sim_data_modified = np.hstack((participantIndexArray, responsesArray, reactionTimesArray, participantStakes))
    overallData = np.vstack((overallData, sim_data_modified))
    
print(np.shape(overallData))

-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('number of trials: ', 224)
-------
('node: ', 'wfpt.1')
('

-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node: ', 'wfpt.12')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node: ', 'wfpt.15')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node: ', 'wfpt.18')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node: ', 'wfpt.20')
('number of trials: ', 224)
-------
('node

...

-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node: ', 'wfpt.31')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node: ', 'wfpt.34')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node: ', 'wfpt.37')
('number of trials: ', 224)
-------
('node

-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('number of trials: ', 224)
-------
('node: ', 'wfpt.4')
('

-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.7')
('number of trials: ', 224)
-------
('node: ', 'wfpt.8')
('number of trials: ', 224)
-------
('node: ', 'wfpt.8')
('number of trials: ', 224)
-------
('node: ', 'wfpt.8')
('number of trials: ', 224)
-------
('node: ', 'wfpt.8')
('

In [12]:
import pandas as pd

import os
if not os.path.exists('simulatedData/{}'.format(modelName)):
    os.makedirs('simulatedData/{}'.format(modelName))


df = pd.DataFrame(overallData[1:, :])
df.columns = ['subj_idx', 'response', 'rt', 'gain', 'loss']
df.to_csv('simulatedData/{}/{}.csv'.format(modelName, modelName))
# np.savetxt("simulatedData.csv", overallData, delimiter=",", fmt='%.5f')